In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install vit_keras

# !pip install tensorflow_addons

In [ ]:
import os
import cv2
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, Input, Lambda, BatchNormalization, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

"""
## Hyperparameters
"""

epochs = 100
batch_size = 32
margin = 1  # Margin for constrastive loss.

In [ ]:
save_weights_path = "/content/drive/MyDrive/Graduation_Project/model_weight_Tiamese_mobile_2.hdf5"
load_weights_path = "/content/drive/MyDrive/Graduation_Project/model_weight_Tiamese_mobile.hdf5"

In [ ]:
def load_data(path,input_shape):
    image_list=[]
    label_list=[]
    classes=['No_Finding', 'Enlarged_Cardiomediastinum', 'Cardiomegaly',
       'Lung_Opacity', 'Lung_Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural_Effusion', 'Pleural_Other',
       'Fracture', 'Support_Devices']
    for category in classes:
        picList= os.listdir(path+"/"+str(category))
        for pic in tqdm(picList):
            # image= cv2.imread(path+"/"+str(category)+"/"+pic)
            # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # image= cv2.resize(image,input_shape)/255.0
            image_list.append(path+"/"+str(category)+"/"+pic)
            label_list.append(classes.index(category))

    image_df = pd.DataFrame({'Path':image_list, 'Label':label_list})

   # return np.array(image_list),np.array(label_list)
    return image_df

In [ ]:
train = load_data('/content/drive/MyDrive/Graduation_Project/Train', (224,224))

100%|██████████| 480/480 [00:00<00:00, 539749.58it/s]


In [ ]:
val = load_data('/content/drive/MyDrive/Graduation_Project/Val', (224,224))

100%|██████████| 120/120 [00:00<00:00, 297996.73it/s]


In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, train_df, input_shape=224, batch_size=32, shuffle=True):
        self.train_df = train_df
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(self.train_df.shape[0] / self.batch_size)

    def __getitem__(self,index):
        'Generate one batch of data'
        pairs, labels = self.__data_generation()
        pairs_1=pairs[:, 0]
        pairs_2=pairs[:, 1]

        return [pairs_1,pairs_2], labels

    
    def make_pairs(self, x, y):
        """Creates a tuple containing image pairs with corresponding label.
        Arguments:
            x: List containing images, each index in this list corresponds to one image.
            y: List containing labels, each label with datatype of `int`.
        Returns:
            Tuple containing two numpy arrays as (pairs_of_samples, labels),
            where pairs_of_samples' shape is (2len(x), 2,n_features_dims) and
            labels are a binary array of shape (2len(x)).
        """

        classes = np.unique(y)
        #digit_indices = [np.where(y == i)[0] for i in classes]

        pairs = []
        labels = []

        for idx1 in range(len(x)):
            # add a matching example
            x1 = x[idx1]
            label1 = y[idx1]
            idx2 = random.choice(np.where(y== label1)[0])
            x2 = x[idx2]

            pairs += [[x1, x2]]
            labels += [1]

            # add a non-matching example
            label2 = random.choice(classes)
            while label2 == label1:
                label2 = random.choice(classes)

            idx2 = random.choice(np.where(y== label2)[0])
            x2 = x[idx2]

            pairs += [[x1, x2]]
            labels += [0]

        return np.array(pairs), np.array(labels).astype("float32")

    def get_images(self, sample_df):
        X = []
        for i in range(sample_df.shape[0]):
            image= cv2.imread(sample_df.iloc[i])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image= cv2.resize(image, (self.input_shape, self.input_shape))/255.0
            X.append(image)
        
        return np.array(X)

    def __data_generation(self):
        'Generates data containing batch_size samples' 
        # Initialization
        indices = np.arange(self.train_df.shape[0])
        idx = np.random.choice(indices, size=self.batch_size)
        X = self.get_images(self.train_df.iloc[idx, 0])
        y = self.train_df.iloc[idx, 1].values
        pairs, labels=self.make_pairs(X,y)

        return pairs, labels

In [ ]:
train_gen = DataGenerator(train, input_shape=224, batch_size=32, shuffle=True)

# make validation pairs
val_gen = DataGenerator(val, input_shape=224, batch_size=32, shuffle=True)

In [ ]:
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.
    Arguments:
        vects: List containing two tensors of same length.
    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

In [ ]:
embedding_network=Sequential()
embedding_network.add(Conv2D(64, (3, 3), activation="relu"))
embedding_network.add(BatchNormalization())
embedding_network.add(AveragePooling2D(pool_size=(2, 2)))
embedding_network.add(Conv2D(32, (3, 3), activation="relu"))
embedding_network.add(BatchNormalization())
embedding_network.add(AveragePooling2D(pool_size=(2, 2)))
embedding_network.add(Flatten())
embedding_network.add(Dense(1024, activation="relu"))



input_1 = Input((224, 224, 3))
input_2 = Input((224, 224, 3))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = BatchNormalization()(merge_layer)
output_layer = Dense(1, activation="sigmoid")(normal_layer)
siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)


In [ ]:
siamese.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 1024)         95573152    ['input_1[0][0]',                
                                                                  'input_2[0][0]']            

In [ ]:
def loss(margin=1):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.
  Arguments:
      margin: Integer, defines the baseline for distance for which pairs
              should be classified as dissimilar. - (default is 1).
  Returns:
      'constrastive_loss' function with data ('margin') attached.
  """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the constrastive loss.
      Arguments:
          y_true: List of labels, each label is of type float32.
          y_pred: List of predictions of same length as of y_true,
                  each label is of type float32.
      Returns:
          A tensor containing constrastive loss as floating point value.
      """

        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

In [ ]:
checkpoint = ModelCheckpoint(save_weights_path, monitor='val_auc', verbose=1, save_best_only=True, mode='max', save_freq='epoch')
early = EarlyStopping(monitor="val_auc", mode='max', patience=10, restore_best_weights=True)
reduceLROnPlato = ReduceLROnPlateau(monitor="val_auc", factor=0.1, patience=5, verbose=1, mode='max')
callbacks_list = [checkpoint, early, reduceLROnPlato]

In [ ]:
siamese.compile(loss=loss(margin=margin), optimizer=Adam(learning_rate=0.1), metrics=["accuracy", tf.keras.metrics.AUC(multi_label = False)])

In [ ]:
history = siamese.fit(train_gen,
    validation_data=val_gen,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks_list
    )

Epoch 1/100
195/195 [==============================] - ETA: 0s - loss: 0.2064 - accuracy: 0.6576 - auc: 0.7041
Epoch 00001: val_auc improved from -inf to 0.72700, saving model to /content/drive/MyDrive/Graduation_Project/model_weight_Tiamese_mobile_2.hdf5
195/195 [==============================] - 138s 683ms/step - loss: 0.2064 - accuracy: 0.6576 - auc: 0.7041 - val_loss: 0.2476 - val_accuracy: 0.6948 - val_auc: 0.7270 - lr: 0.1000
Epoch 2/100
195/195 [==============================] - ETA: 0s - loss: 0.2030 - accuracy: 0.6682 - auc: 0.7078
Epoch 00002: val_auc did not improve from 0.72700
195/195 [==============================] - 123s 630ms/step - loss: 0.2030 - accuracy: 0.6682 - auc: 0.7078 - val_loss: 0.2074 - val_accuracy: 0.6703 - val_auc: 0.7021 - lr: 0.1000
Epoch 3/100
195/195 [==============================] - ETA: 0s - loss: 0.2011 - accuracy: 0.6704 - auc: 0.7082
Epoch 00003: val_auc did not improve from 0.72700
195/195 [==============================] - 123s 629ms/step - l

In [ ]:
def My_inputs(model,dir_image,threshold):
  D_idx={ 'No_Finding':0, 'Enlarged_Cardiomediastinum':1, 'Cardiomegaly':2,
       'Lung_Opacity':3, 'Lung_Lesion':4, 'Edema':5, 'Consolidation':6, 'Pneumonia':7,
       'Atelectasis':8, 'Pneumothorax':9, 'Pleural_Effusion':10, 'Pleural_Other':11,
       'Fracture':12, 'Support_Devices':13}
  to_predict_image=cv2.imread(dir_image)
  to_predict_image=cv2.resize(to_predict_image,(256,256))
  to_predict_image=to_predict_image[np.newaxis,:,:,:]
  prediction=np.zeros((14,1))

  for d in os.listdir('/content/drive/MyDrive/Data/Val'):
    preds=[]

    for img in os.listdir('/content/drive/MyDrive/Data/Val/'+d):
      my_img=cv2.imread('/content/drive/MyDrive/Data/Val/'+d+'/'+img)
      my_img=cv2.resize(my_img,(256,256))
      my_img=my_img[np.newaxis,:,:,:]
      preds.append(model.predict([to_predict_image,my_img]))
    prediction[D_idx[d]]=np.array(preds).mean()
  return prediction 

In [ ]:
R=My_inputs(siamese,'/content/drive/MyDrive/Data/Val/No_Finding/No_Finding_101.jpg',0.2)
R

array([[0.10481144],
       [0.00388822],
       [0.00075505],
       [0.00072143],
       [0.02478264],
       [0.02040445],
       [0.00789811],
       [0.01149207],
       [0.00643973],
       [0.00372105],
       [0.00261437],
       [0.03291192],
       [0.00565439],
       [0.00737246]])

In [ ]:
R2=My_inputs(siamese,'/content/drive/MyDrive/Data/Val/Cardiomegaly/Cardiomegaly_101.jpg',0.2)
R2

array([[0.02367543],
       [0.0159646 ],
       [0.09395885],
       [0.00236193],
       [0.03043787],
       [0.01930635],
       [0.01341991],
       [0.0034225 ],
       [0.0055633 ],
       [0.0011952 ],
       [0.0086239 ],
       [0.03168651],
       [0.003994  ],
       [0.00717427]])

In [ ]:
R3=My_inputs(siamese,'/content/drive/MyDrive/Data/Val/Edema/Edema_107.jpg',0.2)
R3

array([[0.00981312],
       [0.00934001],
       [0.02268913],
       [0.00242263],
       [0.01622439],
       [0.09319084],
       [0.01102469],
       [0.0056631 ],
       [0.00421425],
       [0.00228461],
       [0.0125719 ],
       [0.0513186 ],
       [0.00573691],
       [0.00672107]])

In [ ]:
R4=My_inputs(siamese,'/content/drive/MyDrive/Data/Val/Pneumonia/Pneumonia_101.jpg',0.2)
R4

array([[2.92931870e-03],
       [4.17525735e-04],
       [3.80573329e-05],
       [6.67607132e-03],
       [1.11673505e-03],
       [2.45329551e-03],
       [1.08795666e-05],
       [8.44141468e-02],
       [1.24711054e-03],
       [1.95492612e-04],
       [2.28310615e-04],
       [5.04220650e-03],
       [3.78303463e-03],
       [4.93039470e-03]])

In [ ]:
R5=My_inputs(siamese,'/content/drive/MyDrive/Data/Val/Pleural_Effusion/Pleural_Effusion_102.jpg',0.2)
R5

array([[3.85959429e-04],
       [2.86779774e-04],
       [1.73172171e-04],
       [1.45328013e-04],
       [3.97006836e-04],
       [4.80900344e-04],
       [7.92012579e-05],
       [7.03038677e-05],
       [5.10444923e-04],
       [2.36833894e-05],
       [8.30149353e-02],
       [9.81115620e-04],
       [1.54916255e-03],
       [1.13958587e-04]])